In [0]:
%sql

SELECT DISTINCT SLOT_MNEMONIC FROM 4_prod.raw.mill_sch_appt
WHERE SLOT_MNEMONIC ILIKE '%antenatal%'
LIMIT 100

In [0]:
%sql

SELECT * FROM 3_lookup.mill.mill_code_value
WHERE DESCRIPTION LIKE '%CMV%'

In [0]:
%sql
-- Number of all adult women who received antenatal care between 2019-12-01 and 2024-12-31 
SELECT COUNT(DISTINCT a.PERSON_ID)
FROM 4_prod.raw.mill_sch_appt AS a
INNER JOIN 4_prod.raw.mill_sch_event AS e
ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
INNER JOIN 4_prod.raw.mill_person AS p
ON a.PERSON_ID = p.PERSON_ID
WHERE 
  APPT_SYNONYM_FREE ILIKE '%antenatal%'
  AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
  AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
LIMIT 100

In [0]:
%sql
-- Number of adult women who received antenatal care between 2019-12-01 and 2024-12-31 and were recorded with at least one 'CMV' clinical event
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
WHERE
  EVENT_TITLE_TEXT ILIKE '%CMV%'
  --AND EVENT_START_DT_TM BETWEEN '2019-12-01' AND '2024-12-31' -- got 0 result


In [0]:
%sql
-- Number of adult women who received antenatal care between 2019-12-01 and 2024-12-31 and were recorded with at least one clinical event with CMV-related EVENT_CD in the same period
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- linked with a (PI_CDE) blob content
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID), COUNT(DISTINCT bc.CE_BLOB_CONTENT_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.pi_cde_blob_content AS bc
ON ce.EVENT_ID = bc.EVENT_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- clinical event linked with a (PI_CDE) blob content
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID), COUNT(DISTINCT bc.CE_BLOB_CONTENT_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.pi_cde_blob_content AS bc
ON ce.EVENT_ID = bc.EVENT_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN ante
ON bc.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN ante
  ON ce.PERSON_ID = ante.PERSON_ID
  WHERE
    (cv.DESCRIPTION LIKE '%CMV%' OR cv.DESCRIPTION ILIKE '%Cytomegalovirus%')
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN cmv_ce
ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql

-- Number of adult women who were recorded at least one clinical event with EVENT_CD related to CMV regardless of attending any antenatal appointment
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 4_prod.raw.mill_person AS p
ON ce.PERSON_ID = p.PERSON_ID
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
WHERE 
  cv.DESCRIPTION LIKE '%CMV%'
  AND SEX_CD = 362 -- female
  AND DATEDIFF(YEAR, BIRTH_DT_TM, EVENT_START_DT_TM) >= 18
  AND COALESCE(EVENT_END_DT_TM, EVENT_START_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'

In [0]:
%sql

WITH cohort AS (
  SELECT DISTINCT a.PERSON_ID FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND BIRTH_DT_TM < '2001-12-01'
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  GROUP BY EVENT_ID
)
SELECT b.* 
FROM 4_prod.raw.pi_cde_blob_content AS b
INNER JOIN ce
ON ce.EVENT_ID = b.EVENT_ID
INNER JOIN cohort
ON cohort.PERSON_ID = ce.PERSON_ID
WHERE BLOB_CONTENTS LIKE '%CMV%'

LIMIT 100


In [0]:
%sql

SELECT COUNT(DISTINCT p.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS e
INNER JOIN 4_prod.raw.mill_person AS p
ON e.PERSON_ID = p.PERSON_ID
WHERE 
  EVENT_TITLE_TEXT LIKE '%CMV%'
  AND DATEDIFF(DAY, p.BIRTH_DT_TM, e.EVENT_START_DT_TM) <= 28 

LIMIT 100


In [0]:
%sql
/* 
Number of distinct patients who are
  -- new born <= 28 days
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN 4_prod.raw.mill_person AS p
  ON p.PERSON_ID = ce.PERSON_ID
  WHERE
    cv.DESCRIPTION LIKE '%CMV%'
    AND DATEDIFF(DAY, p.BIRTH_DT_TM, COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM)) <= 28 
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN cmv_ce
ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_person
WHERE BIRTH_DT_TM > '2025-07-01'
LIMIT 10

In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_clinical_event
WHERE EVENT_TITLE_TEXT LIKE '%CMV%'
LIMIT 1000